In [539]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import datetime
from datetime import timedelta
from pytz import timezone
import espn_scraper as espn #https://github.com/andr3w321/espn_scraper
import json

def strzero(num):
    num = str(num)
    if len(num) == 1:
        num = '0'+num
    return(num)

In [540]:
year = 2022
month = 8
day = 22
date = '20000822'
schedURL = 'https://www.espn.com/wnba/scoreboard/_/date/'+str(year)+strzero(month)+strzero(day)
schedURL

'https://www.espn.com/wnba/scoreboard/_/date/20220822'

In [574]:
game_ids = []
game_info = []
years = list(range(2001,2023))
#years = [year]
years.reverse()

In [575]:
years

[2022,
 2021,
 2020,
 2019,
 2018,
 2017,
 2016,
 2015,
 2014,
 2013,
 2012,
 2011,
 2010,
 2009,
 2008,
 2007,
 2006,
 2005,
 2004,
 2003,
 2002,
 2001]

In [576]:
#get all of the game ids ever
for year in years:
    prev_len = len(game_ids)
    scoreboard_urls= espn.get_all_scoreboard_urls('wnba', year) 
    for scoreboard_url in scoreboard_urls:
        data = espn.get_url(scoreboard_url, cached_path="cached_data")
        try:
            for event in data['content']:  
                if event['analytics']['gameId'] not in game_ids:
                    game_ids.append(event['analytics']['gameId'])
                    game_info.append([event['analytics']['gameId'],year])
        except:
            pass
    print([year,len(game_ids)-prev_len])


[2022, 255]
[2021, 218]
[2020, 154]
[2019, 236]
[2018, 237]
https://sportscenter.api.espn.com/apis/v1/events?sport=basketball&league=wnba&dates=20170914
[2017, 234]
[2016, 235]
https://sportscenter.api.espn.com/apis/v1/events?sport=basketball&league=wnba&dates=20150830
[2015, 242]
[2014, 239]
[2013, 236]
[2012, 237]
[2011, 239]
https://sportscenter.api.espn.com/apis/v1/events?sport=basketball&league=wnba&dates=20100810
https://sportscenter.api.espn.com/apis/v1/events?sport=basketball&league=wnba&dates=20100812
https://sportscenter.api.espn.com/apis/v1/events?sport=basketball&league=wnba&dates=20100813
https://sportscenter.api.espn.com/apis/v1/events?sport=basketball&league=wnba&dates=20100814
https://sportscenter.api.espn.com/apis/v1/events?sport=basketball&league=wnba&dates=20100815
https://sportscenter.api.espn.com/apis/v1/events?sport=basketball&league=wnba&dates=20100817
https://sportscenter.api.espn.com/apis/v1/events?sport=basketball&league=wnba&dates=20100820
https://sportscente

In [577]:
game_ids

['401430475',
 '401430476',
 '401430477',
 '401430478',
 '401430479',
 '401430480',
 '401430481',
 '401430482',
 '401430483',
 '401430484',
 '401391650',
 '401391651',
 '401391652',
 '401391653',
 '401391654',
 '401391655',
 '401391656',
 '401391658',
 '401391657',
 '401391659',
 '401391660',
 '401391661',
 '401391662',
 '401391663',
 '401391664',
 '401391665',
 '401391666',
 '401391667',
 '401391668',
 '401391669',
 '401391670',
 '401391671',
 '401391672',
 '401391673',
 '401391674',
 '401391675',
 '401391676',
 '401391677',
 '401391678',
 '401391679',
 '401391680',
 '401391681',
 '401391682',
 '401391683',
 '401391684',
 '401391685',
 '401391686',
 '401391687',
 '401391688',
 '401391689',
 '401391690',
 '401391691',
 '401391692',
 '401391693',
 '401391694',
 '401391695',
 '401391696',
 '401391697',
 '401391698',
 '401391699',
 '401391700',
 '401391701',
 '401391702',
 '401391703',
 '401391704',
 '401391705',
 '401391706',
 '401391707',
 '401391708',
 '401391709',
 '401391710',
 '4013

In [598]:
len(game_ids)

5433

In [569]:
#game_ids_csv = pd.read_csv('not-scraped.csv')
GameID = '401391720'

In [580]:
#def get_game_data(GameID):
if True:
    GameID = str(GameID)

    #scrape html
    URL = 'https://www.espn.com/wnba/playbyplay/_/gameId/'+GameID
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #extract json including pbp data
    #from: https://stackoverflow.com/questions/13323976/how-to-extract-a-json-object-that-was-defined-in-a-html-page-javascript-block-us
    script = soup.find('script', text=re.compile("window\['__espnfitt__'\]"))
    json_text = re.search(r"^\s*window\['__espnfitt__'\]\s*=\s*({.*?})\s*;\s*$",
                          script.string, flags=re.DOTALL | re.MULTILINE).group(1)
    data = json.loads(json_text)
    
    #get pbp
    pbp = data['page']['content']['gamepackage']['pbp']['playGrps']
    pbp = [item for sublist in pbp for item in sublist]
    pbp = pd.DataFrame.from_dict(pbp)
    
    try:
        #get shot chart
        shot_chart = data['page']['content']['gamepackage']['shtChrt']['plays']
        shot_chart = pd.DataFrame.from_dict(shot_chart)
        shot_chart = shot_chart[['id','athlete','coordinate','shootingPlay','type']]
        
        #get neutral site
        is_neutral = data['page']['content']['gamepackage']['shtChrt']['ntrlSte']
    
        #combine shot chart and pbp
        combined_pbp = pd.merge(pbp,shot_chart,on='id',how='outer')
        
    except:
        combined_pbp = pbp
        combined_pbp[['id','athlete','coordinate','shootingPlay','type']] = np.nan
        is_neutral = np.nan
    
    #clean combined dataframe
    combined_pbp['period'] = [re.search('[0-9]',str(x)).group(0) for x in combined_pbp['period']]
    combined_pbp['clock'] = [re.search('[0-9]{1,2}([:\.])[0-9]{1,2}',str(x)).group(0) for x in combined_pbp['clock']]
    combined_pbp['player_id'] = [re.search('\d+',str(x)).group(0) if bool(re.search('\d+',str(x))) else np.nan for x in combined_pbp['athlete']]
    combined_pbp['player'] = [re.sub("{'id': '\d+', 'name': '",'',str(x)) if bool(re.search("{'id': '\d+', 'name': '",str(x))) else np.nan for x in combined_pbp['athlete']]
    combined_pbp['player'] = [re.sub("'}",'',str(x)) if bool(re.search("'}",str(x))) else np.nan for x in combined_pbp['player']]
    combined_pbp['shotType'] = [re.sub("{'id': '\d+', 'txt': '",'',str(x)) if bool(re.search("{'id': '\d+', 'txt': '",str(x))) else np.nan for x in combined_pbp['type']]
    combined_pbp['shotType'] = [re.sub("'}",'',str(x)) if bool(re.search("'}",str(x))) else np.nan for x in combined_pbp['shotType']]
    combined_pbp['shotType'] = [re.sub("{'id': '\d+', 'txt': '",'',str(x)) if bool(re.search("{'id': '\d+', 'txt': '",str(x))) else np.nan for x in combined_pbp['type']]
    combined_pbp['shotType'] = [re.sub("'}",'',str(x)) if bool(re.search("'}",str(x))) else np.nan for x in combined_pbp['shotType']]
    combined_pbp['x_coord'] = [re.findall("\d+",str(x))[0] if bool(re.search("\d+",str(x))) else np.nan for x in combined_pbp['coordinate']]
    combined_pbp['y_coord'] = [re.findall("\d+",str(x))[1] if bool(re.search("\d+",str(x))) else np.nan for x in combined_pbp['coordinate']]
    combined_pbp = combined_pbp.drop('athlete', axis=1)
    combined_pbp = combined_pbp.drop('type', axis=1)
    combined_pbp = combined_pbp.drop('coordinate', axis=1)
    
    #team info
    teams = data['page']['content']['gamepackage']['pbp']['tms']
    combined_pbp['homeTeam'] = teams['home']['abbrev']
    combined_pbp['awayTeam'] = teams['away']['abbrev']
    if 'winner' in teams['home']:
        combined_pbp['winner'] = 'home'
    elif 'winner' in teams['away']:
        combined_pbp['winner'] = 'away'
    
    #game info
    game_info = data['page']['content']['gamepackage']['gmInfo']
    combined_pbp['gameID'] = GameID
    combined_pbp['dateTime'] = datetime.datetime.strptime(game_info['dtTm'],'%Y-%m-%dT%H:%MZ') - timedelta(hours=7) #put datetime in pacific time
    combined_pbp['neutralSite'] = is_neutral
    
    #return combined_pbp

In [548]:
df = pd.DataFrame()
gid_fails = []
#game_ids_test = [401230733]
game_ids_test = game_ids
for GameID in game_ids_test:
    try:
        temp = get_game_data(GameID) #for each game id get pbp, shot chart, game info in one table
        df = pd.concat([df,temp],ignore_index=True) #append to previous
    except:
        gid_fails.append(GameID) #if fail, print the gameid
        
print(gid_fails)

['401430480', '401455627', '401455628', '401455629', '401455650', '401455651', '401455652', '401455653', '401455654', '401455656', '401455657', '401455658', '401455659', '401455660', '401455661', '401455671', '401455674', '401455675', '401455676', '401455677', '401455678', '401455679', '401455680', '401455681', '401455682', '401455683', '401455684', '401455685']


In [549]:
game_ids_test = gid_fails
gid_fails = []
for GameID in game_ids_test:
    try:
        temp = get_game_data(GameID) #for each game id get pbp, shot chart, game info in one table
        df = pd.concat([df,temp],ignore_index=True) #append to previous
    except:
        gid_fails.append(GameID) #if fail, print the gameid
        
print(gid_fails)

['401430480', '401455627', '401455628', '401455629', '401455650', '401455651', '401455652', '401455653', '401455654', '401455656', '401455657', '401455658', '401455659', '401455660', '401455661', '401455671', '401455674', '401455675', '401455676', '401455677', '401455678', '401455679', '401455680', '401455681', '401455682', '401455683', '401455684', '401455685']


In [550]:
df.to_csv('data/raw-data/espn-raw-pbp-'+str(year)+'.csv',index=False)

In [551]:
sum(df['gameID'] == '290626014')

0

In [552]:
df

,id,period,text,homeAway,awayScore,homeScore,clock,scoringPlay,shootingPlay,player_id,...,y_coord,homeTeam,awayTeam,winner,gameID,dateTime,neutralSite,isFoul,isOT,otNum
0,4014304754,1,Jordin Canada vs. Breanna Stewart (Jordin Cana...,away,0,0,10:00,NaN,NaN,NaN,...,NaN,SEA,LA,home,401430475,2022-04-23 18:00:00,False,NaN,NaN,NaN
1,4014304757,1,Epiphanny Prince blocks Jordin Canada 's 14-fo...,away,0,0,9:38,NaN,True,3142250,...,11,SEA,LA,home,401430475,2022-04-23 18:00:00,False,NaN,NaN,NaN
2,4014304759,1,Breanna Stewart defensive rebound,home,0,0,9:35,NaN,NaN,NaN,...,NaN,SEA,LA,home,401430475,2022-04-23 18:00:00,False,NaN,NaN,NaN
3,40143047510,1,Breanna Stewart misses 25-foot three point pul...,home,0,0,9:29,NaN,True,2998928,...,24,SEA,LA,home,401430475,2022-04-23 18:00:00,False,NaN,NaN,NaN
4,40143047511,1,Nneka Ogwumike defensive rebound,away,0,0,9:29,NaN,NaN,NaN,...,NaN,SEA,LA,home,401430475,2022-04-23 18:00:00,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88372,401391865548,4,Veronica Burton defensive rebound,away,116,86,9.4,NaN,NaN,NaN,...,NaN,LA,DAL,away,401391865,2022-08-14 16:00:00,False,NaN,NaN,NaN
88373,401391865549,4,Veronica Burton steps out of bounds turnover,away,116,86,9.4,NaN,NaN,NaN,...,NaN,LA,DAL,away,401391865,2022-08-14 16:00:00,False,NaN,NaN,NaN
88374,401391865550,4,Jasmine Walker makes 18-foot jumper (Jordin Ca...,home,116,88,7.0,True,True,4065832,...,14,LA,DAL,away,401391865,2022-08-14 16:00:00,False,NaN,NaN,NaN
88375,401391865552,4,End of the 4th Quarter,NaN,116,88,0.0,NaN,NaN,NaN,...,NaN,LA,DAL,away,401391865,2022-08-14 16:00:00,False,NaN,NaN,NaN


In [582]:
#df[['gameID','dateTime','homeTeam','awayTeam']].drop_duplicates().to_csv(year+"-scraped.csv")


2

In [554]:
not_scraped = pd.read_csv('data/raw-data/not-scraped.csv')
current_not_scraped = pd.DataFrame(gid_fails,columns=['gid'])
current_not_scraped['year'] = year
not_scraped = pd.concat([not_scraped,current_not_scraped])
not_scraped = not_scraped.drop_duplicates(ignore_index=True)
not_scraped.to_csv('data/raw-data/not-scraped.csv',index=False)

In [585]:
#to do 
#go back and get other game info
all_season_types = {}
for GameID in game_ids:
    GameID = str(GameID)

    try:
        #scrape html
        URL = 'https://www.espn.com/wnba/playbyplay/_/gameId/'+GameID
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        #extract json including pbp data
        #from: https://stackoverflow.com/questions/13323976/how-to-extract-a-json-object-that-was-defined-in-a-html-page-javascript-block-us
        script = soup.find('script', text=re.compile("window\['__espnfitt__'\]"))
        json_text = re.search(r"^\s*window\['__espnfitt__'\]\s*=\s*({.*?})\s*;\s*$",
                              script.string, flags=re.DOTALL | re.MULTILINE).group(1)
        data = json.loads(json_text)
        
        #get 
        season_type = data['page']['content']['gamepackage']['gmStrp']['seasonType']
        all_season_types[GameID] = season_type
    except: 
        all_season_types[GameID] = np.nan
        

In [601]:
pd.DataFrame.from_dict({'GameID':list(all_season_types.keys()),'seasonType':list(all_season_types.values())}).to_csv('data/raw-data/gametypes.csv',index=False)

In [556]:
##archive parse html code not needed with ESPN parsing as of 7/30/22
##if True:
#def get_game_data(GameID):
#    #first get all the game info
#    data_type = "recap"
#    url = espn.get_game_url(data_type, "wnba", GameID)
#    data = espn.get_url(url)
#    temp = data['gamepackageJSON']
#    temp = temp['header']   
#    #date
#    Date = temp['competitions'][0]['date']
#    #winning team
#    if temp['competitions'][0]['competitors'][0]['winner']: #check if 1st team won
#        WinningTeam = temp['competitions'][0]['competitors'][0]['team']['abbreviation']
#    else: #otehrwise 2nd team won
#        WinningTeam = temp['competitions'][0]['competitors'][1]['team']['abbreviation']
#    #regular season or playoffs?
#    gametypes = ['none','preseason','regular season','playoffs']
#    gametype_int = temp['season']['type']
#    GameType = gametypes[gametype_int]
#    
#    ########################################
#    #now get all the play-by-play
#    GameID = str(GameID)
#    URL = 'https://www.espn.com/wnba/playbyplay?gameId='+GameID
#    page = requests.get(URL)
#    soup = BeautifulSoup(page.content, 'html.parser')
#    
#    #teams
#    teams = [x.text for x in soup.find_all('span', class_='abbrev')]
#    AwayTeam = teams[0]
#    HomeTeam = teams[1]
#    teams = {teams[0]:'away',teams[1]:'home'}
#    
#    #scrape the shot description table
#    times = [x.text for x in soup.find_all('td', class_='time-stamp')]
#    plays = [x.text for x in soup.find_all('td', class_='game-details')]
#    score = [x.text for x in soup.find_all('td', class_='combined-score')]
#    
#    #get team names from the logo image urls
#    logourl = [x.find('img').get('src') for x in soup.find_all('td', class_='logo')]
#    team = [re.match(r"https.*/wnba/500/(?P<team>[a-z]+)\.",x).group('team').upper() for x in logourl]
#    homeoraway = [teams[x] for x in team]
#    
#    #convert to df
#    shotdescriptions = pd.DataFrame({'Time': times,
#                          'EventDescription': plays,
#                          'Score': score,
#                          'EventTeam': team,
#                          'HomeOrAway':homeoraway})
#    shotdescriptions['GameID'] = GameID
#    shotdescriptions['Date'] = Date
#    shotdescriptions['GameType'] = GameType
#    shotdescriptions['WinningTeam'] = WinningTeam
#    shotdescriptions['AwayTeam'] = AwayTeam
#    shotdescriptions['HomeTeam'] = HomeTeam
#    
#    
#    #get shot chart data for away team
#    awayshots = soup.find('ul', class_='shots away-team')
#    awayshots = awayshots.find_all('li')
#    awayshotchart = []
#    for row in awayshots:
#        coordtext = row.get('style')
#        match = re.match(r".*left:(?P<xcoord>[0-9\.]+)%;top:(?P<ycoord>[0-9\.]+)%;",coordtext)
#        xcoord = match.group('xcoord')
#        ycoord = match.group('ycoord')
#        ycoord = str(100-float(ycoord))
#        row_data = [row.get('class')[0],
#                    row.get('data-homeaway'),
#                    row.get('data-period'),
#                    row.get('data-shooter'),
#                    row.get('data-text'),
#                    row.get('id'),
#                    xcoord,
#                    ycoord]
#        awayshotchart.append(row_data)
#    
#    #get shot chart data for home team
#    homeshots = soup.find('ul', class_='shots home-team')
#    homeshots = homeshots.find_all('li')
#    homeshotchart = []
#    for row in homeshots:
#        coordtext = row.get('style')
#        match = re.match(r".*left:(?P<xcoord>[0-9\.]+)%;top:(?P<ycoord>[0-9\.]+)%;",coordtext)
#        xcoord = match.group('xcoord')
#        ycoord = match.group('ycoord')
#        ycoord = str(100-float(ycoord))
#        row_data = [row.get('class')[0],
#                    row.get('data-homeaway'),
#                    row.get('data-period'),
#                    row.get('data-shooter'),
#                    row.get('data-text'),
#                    row.get('id'),
#                    xcoord,
#                    ycoord]
#        homeshotchart.append(row_data)
#    
#    awayshotchart = pd.DataFrame(awayshotchart)
#    homeshotchart = pd.DataFrame(homeshotchart)
#    
#    #name columns for shot chart data
#    awayshotchart.columns = ['ShotOutcome','EventTeam','Quarter','pid','EventDescription','ShotID','xcoord','ycoord']
#    homeshotchart.columns = ['ShotOutcome','EventTeam','Quarter','pid','EventDescription','ShotID','xcoord','ycoord']
#    
#    #join to shot descriptions
#    playindex = 0
#    for row in range(len(awayshotchart)):
#        not_found = True
#        while not_found:
#            if shotdescriptions.loc[playindex,'EventDescription']==awayshotchart.loc[row,'EventDescription']:
#                if shotdescriptions.loc[playindex,'HomeOrAway']=='away':
#                    shotdescriptions.loc[playindex,'ShotOutcome']=awayshotchart.loc[row,'ShotOutcome']
#                    shotdescriptions.loc[playindex,'Quarter']=awayshotchart.loc[row,'Quarter']
#                    shotdescriptions.loc[playindex,'pid']=awayshotchart.loc[row,'pid']
#                    shotdescriptions.loc[playindex,'ShotID']=awayshotchart.loc[row,'ShotID']
#                    shotdescriptions.loc[playindex,'xcoord']=awayshotchart.loc[row,'xcoord']
#                    shotdescriptions.loc[playindex,'ycoord']=awayshotchart.loc[row,'ycoord']
#                    not_found = False
#            playindex += 1       
#    
#    playindex = 0
#    for row in range(len(homeshotchart)):
#        not_found = True
#        while not_found:
#            if shotdescriptions.loc[playindex,'EventDescription']==homeshotchart.loc[row,'EventDescription']:
#                if shotdescriptions.loc[playindex,'HomeOrAway']=='home':
#                    shotdescriptions.loc[playindex,'ShotOutcome']=homeshotchart.loc[row,'ShotOutcome']
#                    shotdescriptions.loc[playindex,'Quarter']=homeshotchart.loc[row,'Quarter']
#                    shotdescriptions.loc[playindex,'pid']=homeshotchart.loc[row,'pid']
#                    shotdescriptions.loc[playindex,'ShotID']=homeshotchart.loc[row,'ShotID']
#                    shotdescriptions.loc[playindex,'xcoord']=homeshotchart.loc[row,'xcoord']
#                    shotdescriptions.loc[playindex,'ycoord']=homeshotchart.loc[row,'ycoord']
#                    not_found = False
#            playindex += 1      
#    
#    #get quarter numbers and scores
#    quarter = 1
#    shotdescriptions.loc[0,'AwayScore'] = 0
#    shotdescriptions.loc[0,'HomeScore'] = 0
#    for row in range(0,len(shotdescriptions)):
#        if re.match('End of Game',shotdescriptions.loc[row,'EventDescription']) :
#            quarter -= 1 #make the end of the game the same as the last quarter
#        shotdescriptions.loc[row,'Quarter'] = quarter
#        match = re.match(("([0-9]+) - ([0-9]+)"),shotdescriptions.loc[row,'Score'])
#        shotdescriptions.loc[row,'AwayScore'] = match.group(1)
#        shotdescriptions.loc[row,'HomeScore'] = match.group(2)
#        if re.match('End',shotdescriptions.loc[row,'EventDescription']) :
#            quarter += 1 #if you reach the end of the quarter, go to the next quarter
#    
#    #return shotdescriptions

In [557]:
#url = espn.get_game_url('playbyplay', "wnba", 401244185)
#data = espn.get_url(url)